In [1]:
import anndata as ad 

In [2]:
state_Data = ad.read_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/replogle/total_data_adata.h5ad")
#squidiff_Data = ad.read_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/diff_data/diff_train.h5ad")
import toml 
config = toml.load("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/replogle/hepg2.toml")

In [3]:
import pickle 
file = pickle.load(open("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours_small_test_total2k_gene/vocab_state.pkl","rb"))

In [4]:
import sample_squidiff_ours
sampler = sample_squidiff_ours.sampler(
        model_path = '/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours_small_test_total2k_gene/model.pt',
        perturb_len = len(file['perturb']),
        batch_len = len(file['batch']),
        cell_line_len = len(file['cell_line']),
        gene_size = sum(state_Data.var['highly_variable']),
        #gene_size=201,
        diffusion_steps = 1000
    )

load model and diffusion...
{'output_dim': 2000, 'num_layers': 3, 'num_channels': 128, 'dropout': 0.0, 'class_cond': False, 'use_checkpoint': False, 'use_scale_shift_norm': True, 'use_fp16': False, 'use_encoder': True, 'state_dataset_config': {'perturb_len': 11, 'batch_len': 56, 'cell_line_len': 4, 'gene_size': 2000, 'output_dim': 2000}, 'learn_sigma': False, 'diffusion_steps': 1000, 'noise_schedule': 'linear', 'timestep_respacing': '', 'use_kl': False, 'predict_xstart': False, 'rescale_timesteps': False, 'rescale_learned_sigmas': False, 'data_path': '', 'schedule_sampler': 'uniform', 'lr': 0.0001, 'weight_decay': 0.0, 'lr_anneal_steps': 100000.0, 'batch_size': 16, 'microbatch': -1, 'ema_rate': '0.9999', 'log_interval': 10000.0, 'save_interval': 10000.0, 'resume_checkpoint': '', 'fp16_scale_growth': 0.001, 'use_ddim': True, 'logger_path': '', 'model_path': '/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours_small_test_total2k_gene/model.pt', 'comb_num': 1, 'drug_dimension'

RuntimeError: Error(s) in loading state_dict for our_MLPModel:
	Missing key(s) in state_dict: "encoder.pert_encoder.weight", "encoder.batch_encoder.weight". 

In [7]:
import numpy as np 
def one_hot(label, key):
    vec = np.zeros(len(file[key]), dtype=int)
    idx = file[key][label]
    vec[idx] = 1
    return vec

In [6]:
test_gene = config['fewshot']['replogle_proper.hepg2']['val']+config['fewshot']['replogle_proper.hepg2']['test']

In [7]:
state_Data = state_Data[state_Data.obs['gene'].isin(file['perturb'])]

In [8]:
state_Data.obs['perturb_label'] = state_Data.obs['gene'].apply(lambda x: file['perturb'][x])
state_Data.obs['batch_label'] = state_Data.obs['gem_group'].apply(lambda x: file['batch'][x])
state_Data.obs['cell_line_label'] = state_Data.obs['cell_line'].apply(lambda x: file['cell_line'][x])

/tmp/ipykernel_273402/1927203975.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  state_Data.obs['perturb_label'] = state_Data.obs['gene'].apply(lambda x: file['perturb'][x])


In [9]:
cell_type_list = ['jurkat', 'k562', 'rpe1', 'hepg2']

In [10]:
cell_type_list

['jurkat', 'k562', 'rpe1', 'hepg2']

In [17]:
case = case[case.obs['cell_line']=='k562']

In [12]:
case = state_Data

In [269]:
test_data = state_Data[(state_Data.obs['cell_line']=='hepg2')&(state_Data.obs['gene'].isin(test_gene))]

In [54]:
import torch
import numpy as np
z_sem = sampler.model.encoder.forward(
    torch.tensor(case[:, case.var['highly_variable']].X.toarray()).to('cuda'),
    perturb_label = torch.tensor(np.stack(case.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
    batch = torch.tensor(np.stack(case.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
    cell_type = torch.tensor(np.stack(case.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
)

scrnas_pred = sampler.pred(z_sem, gene_size = sum(state_Data.var['highly_variable']))
     

In [55]:
scrnas_pred[0].mean()

tensor(0.1661, device='cuda:0')

In [56]:
r2_, pearsonr_ = sampler.cal_metric(scrnas_pred, case[:, case.var['highly_variable']] )

In [60]:
gt = case[:, case.var['highly_variable']].X.toarray()

In [ ]:
sum(gt.mean(axis=0))

837.3630208335817

: 

: 

: 

In [39]:
import torch

def mmd_loss(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    计算源域数据和目标域数据之间的 MMD Loss (基于高斯核)。
    
    参数:
        source: 源域数据，形状为 (n, d)
        target: 目标域数据，形状为 (m, d)
        kernel_mul: 计算每个高斯核带宽时的乘数
        kernel_num: 高斯核的数量
        fix_sigma: 是否固定 sigma，如果为 None 则根据数据动态计算
    
    返回:
        loss: MMD loss 值
    """
    source, target = torch.tensor(source), torch.tensor(target)
    n = source.size(0)
    m = target.size(0)
    
    # 拼接数据以统一计算核矩阵
    total = torch.cat([source, target], dim=0)
    total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    
    # 计算 L2 距离矩阵
    L2_distance = ((total0 - total1) ** 2).sum(2)
    
    # 计算带宽 (bandwidth)
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n + m)**2
        
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    
    # 计算多核高斯核矩阵
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
    kernels = sum(kernel_val)
    
    # 根据 MMD 公式组合核矩阵部分
    XX = kernels[:n, :n]
    YY = kernels[n:, n:]
    XY = kernels[:n, n:]
    YX = kernels[n:, :n]
    
    loss = torch.mean(XX) + torch.mean(YY) - torch.mean(XY) - torch.mean(YX)
    return loss


In [52]:
scrnas_pred.mean(axis=0)

tensor([0.1020, 0.1203, 0.0708,  ..., 2.0347, 1.2930, 0.8003], device='cuda:0')

In [45]:
mmd_loss(scrnas_pred.detach().cpu().numpy(), case[:, case.var['highly_variable']].X.toarray())

tensor(0.8558)

In [49]:
case[:, case.var['highly_variable']].X.toarray().mean(axis=0)

array([0.13234767, 0.2069115 , 0.10845941, ..., 4.7794104 , 2.974942  ,
       1.8232641 ], dtype=float32)

In [38]:
r2_, pearsonr_

(-1.4091057777404785, 0.9909339)

In [177]:
#拿ttf2作为例子
import torch
perturb, control_label =  "RPS8", "non-targeting"
perturb_state_data = state_Data[state_Data.obs['gene'] == perturb]
perturb_direction_dict = {}
perturb_profile_dict = {}
control_profile_dict = {}
z_perturb_dict = {}
z_control_dict = {}
from tqdm import tqdm 
for cell_type in tqdm(cell_type_list, position=0):
    print(f"Processing cell type: {cell_type}")
    celltype_perturb_Data = perturb_state_data[perturb_state_data.obs['cell_line'] == cell_type]
    control_Data = state_Data[(state_Data.obs['gene'] == control_label)&(state_Data.obs['cell_line'] == cell_type)]
    print(f"Cell type: {cell_type}, Perturbation data shape: {celltype_perturb_Data.shape}, Control data shape: {control_Data.shape}")
    if celltype_perturb_Data.shape[0] == 0 or control_Data.shape[0] == 0:
        print(f"Skipping cell type: {cell_type} due to insufficient data.")
        continue
    z_perturb = sampler.model.encoder.forward(
        x_start = torch.tensor(celltype_perturb_Data[:, state_Data.var['highly_variable']].X.toarray(), dtype=torch.float32).to('cuda'),
        perturb_label = torch.tensor(np.stack(celltype_perturb_Data.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        batch = torch.tensor(np.stack(celltype_perturb_Data.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
        cell_type = torch.tensor(np.stack(celltype_perturb_Data.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
    )
    print(f"Cell type: {cell_type}, Perturbation latent shape: {z_perturb.shape}")
    z_control = sampler.model.encoder.forward(
        x_start = torch.tensor(control_Data[:, state_Data.var['highly_variable']].X.toarray(), dtype=torch.float32).to('cuda'),
        perturb_label = torch.tensor(np.stack(control_Data.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        batch = torch.tensor(np.stack(control_Data.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
        cell_type = torch.tensor(np.stack(control_Data.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
    )
    print(f"Cell type: {cell_type}, Perturbation latent shape: {z_perturb.shape}, Control latent shape: {z_control.shape}")
    direction = z_perturb.mean(axis=0) - z_control.mean(axis=0)
    print(f"Cell type: {cell_type}, Direction shape: {direction.shape}")
    perturb_direction_dict[cell_type] = direction
    z_perturb_dict[cell_type] = z_perturb
    z_control_dict[cell_type] = z_control
    perturb_profile_dict[cell_type] = celltype_perturb_Data
    control_profile_dict[cell_type] = control_Data
    



  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 54.85it/s]

Processing cell type: jurkat
Cell type: jurkat, Perturbation data shape: (96, 6642), Control data shape: (200, 6642)
Cell type: jurkat, Perturbation latent shape: torch.Size([96, 60])
Cell type: jurkat, Perturbation latent shape: torch.Size([96, 60]), Control latent shape: torch.Size([200, 60])
Cell type: jurkat, Direction shape: torch.Size([60])
Processing cell type: k562
Cell type: k562, Perturbation data shape: (0, 6642), Control data shape: (200, 6642)
Skipping cell type: k562 due to insufficient data.
Processing cell type: rpe1
Cell type: rpe1, Perturbation data shape: (30, 6642), Control data shape: (200, 6642)
Cell type: rpe1, Perturbation latent shape: torch.Size([30, 60])
Cell type: rpe1, Perturbation latent shape: torch.Size([30, 60]), Control latent shape: torch.Size([200, 60])
Cell type: rpe1, Direction shape: torch.Size([60])
Processing cell type: hepg2
Cell type: hepg2, Perturbation data shape: (54, 6642), Control data shape: (200, 6642)
Cell type: hepg2, Perturbation lat

In [180]:
z_perturb_dict['rpe1'].shape, z_perturb_dict['jurkat'].shape

(torch.Size([30, 60]), torch.Size([96, 60]))

In [ ]:
# 看看latent在k562上的重建效果
sample_interp_k562_perturb = sampler.pred(
    z_sem = z_control_dict['rpe1'],
    gene_size = sum(state_Data.var['highly_variable'])
)
r2_, pearsonr_ = sampler.cal_metric(sample_interp_k562_perturb,control_profile_dict['rpe1'][:, state_Data.var['highly_variable']] )
# import MMD distance 


In [240]:
import torch

def mmd_loss(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    计算源域数据和目标域数据之间的 MMD Loss (基于高斯核)。
    
    参数:
        source: 源域数据，形状为 (n, d)
        target: 目标域数据，形状为 (m, d)
        kernel_mul: 计算每个高斯核带宽时的乘数
        kernel_num: 高斯核的数量
        fix_sigma: 是否固定 sigma，如果为 None 则根据数据动态计算
    
    返回:
        loss: MMD loss 值
    """
    source, target = torch.tensor(source), torch.tensor(target)
    n = source.size(0)
    m = target.size(0)
    
    # 拼接数据以统一计算核矩阵
    total = torch.cat([source, target], dim=0)
    total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    
    # 计算 L2 距离矩阵
    L2_distance = ((total0 - total1) ** 2).sum(2)
    
    # 计算带宽 (bandwidth)
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n + m)**2
        
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    
    # 计算多核高斯核矩阵
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
    kernels = sum(kernel_val)
    
    # 根据 MMD 公式组合核矩阵部分
    XX = kernels[:n, :n]
    YY = kernels[n:, n:]
    XY = kernels[:n, n:]
    YX = kernels[n:, :n]
    
    loss = torch.mean(XX) + torch.mean(YY) - torch.mean(XY) - torch.mean(YX)
    return loss


In [237]:
x, y  = sample_interp_k562_perturb.to('cpu').detach().numpy(),control_profile_dict['rpe1'][:, state_Data.var['highly_variable']].X.toarray()

In [ ]:
from sklearn.metrics import r2_score 
r2_score(
    control_profile_dict['k562'][:, state_Data.var['highly_variable']].X.toarray().flatten(),
    sample_interp_k562_perturb.cpu().numpy().flatten()
)

In [ ]:
import random 
test_data_gene1 = test_data[test_data.obs['gene']=='']
z_control = z_control_dict['hepg2'][random.sample(range(len(z_control_dict['hepg2'])), test_data_gene1.shape[0]),:]

In [255]:
sample_comb_list = {item: sampler.interp_with_direction(z_sem_origin = z_control_dict['hepg2'],
                                       gene_size = sum(state_Data.var['highly_variable']),
                                       direction = perturb_direction_dict[item],
                                       scale = 1,
                                       add_noise_term = False #add noise if for interpolation
                                      ) for item in cell_type_list if item in perturb_direction_dict.keys()}


In [ ]:
from numpy import mean
from sklearn.metrics import r2_score
y = test_data_gene1[:, state_Data.var['highly_variable']].X.toarray()
for cell_type in sample_comb_list.keys():
    x = sample_comb_list[cell_type].to('cpu').detach().numpy()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
for cell_type in sample_comb_list.keys():
    x = perturb_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
for cell_type in sample_comb_list.keys():
    x = control_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")

Cell type: jurkat, R2 Score: 0.8975968360900879, MMD Loss: 0.8553460836410522
Cell type: rpe1, R2 Score: 0.9010374546051025, MMD Loss: 0.8387625217437744
Cell type: hepg2, R2 Score: 0.9682509899139404, MMD Loss: 0.7014234066009521
Cell type: jurkat, R2 Score: 0.22132223844528198, MMD Loss: 1.0694314241409302
Cell type: rpe1, R2 Score: 0.3465309143066406, MMD Loss: 1.0068998336791992
Cell type: hepg2, R2 Score: 1.0, MMD Loss: 0.0
Cell type: jurkat, R2 Score: 0.028379380702972412, MMD Loss: 1.2870757579803467
Cell type: rpe1, R2 Score: 0.1427953839302063, MMD Loss: 1.1876399517059326
Cell type: hepg2, R2 Score: 0.7867269515991211, MMD Loss: 0.4243168830871582


In [260]:
from numpy import mean
from sklearn.metrics import r2_score
y = control_profile_dict['hepg2'][:, state_Data.var['highly_variable']].X.toarray()
print("Compute distance between control data and generated data")
for cell_type in sample_comb_list.keys():
    x = sample_comb_list[cell_type].to('cpu').detach().numpy()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
print("Compute distance between control data and actual perturbation data")
for cell_type in sample_comb_list.keys():
    x = perturb_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
print("Compute distance between control data and control data")
for cell_type in sample_comb_list.keys():
    x = control_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")

Compute distance between control data and generated data
Cell type: jurkat, R2 Score: 0.9081990718841553, MMD Loss: 0.8784312009811401
Cell type: rpe1, R2 Score: 0.8147814273834229, MMD Loss: 1.1360160112380981
Cell type: hepg2, R2 Score: 0.8645438551902771, MMD Loss: 1.04106605052948
Compute distance between control data and actual perturbation data
Cell type: jurkat, R2 Score: 0.3875315189361572, MMD Loss: 0.9903566837310791
Cell type: rpe1, R2 Score: 0.3738820552825928, MMD Loss: 1.0715481042861938
Cell type: hepg2, R2 Score: 0.8290762901306152, MMD Loss: 0.4243168830871582
Compute distance between control data and control data
Cell type: jurkat, R2 Score: 0.43679553270339966, MMD Loss: 0.9599792957305908
Cell type: rpe1, R2 Score: 0.5377326011657715, MMD Loss: 0.8348215818405151
Cell type: hepg2, R2 Score: 1.0, MMD Loss: 0.0


In [262]:
pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/test_replogle/hepg2_holdout/eval_last.ckpt/adata_pred.h5ad')
truth = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/test_replogle/hepg2_holdout/eval_last.ckpt/adata_real.h5ad')

In [268]:
truth.X

array([[0.77058756, 0.        , 0.        , ..., 3.2787578 , 0.77058756,
        4.4716063 ],
       [0.        , 1.305768  , 0.5143976 , ..., 2.716365  , 2.1281052 ,
        4.0400352 ],
       [0.6739177 , 0.27811354, 0.49547172, ..., 2.4766753 , 1.0728899 ,
        3.9258983 ],
       ...,
       [0.45875692, 0.        , 0.45875692, ..., 2.850647  , 1.5024396 ,
        4.433684  ],
       [0.74850327, 0.74850327, 0.44270754, ..., 2.9924858 , 1.1717597 ,
        4.1920652 ],
       [1.0784274 , 0.6780468 , 1.0784274 , ..., 2.8616548 , 0.6780468 ,
        4.4472556 ]], dtype=float32)

In [ ]:
from cell_eval import MetricsEvaluator
evaluator = MetricsEvaluator(
    adata_pred=pred_ct,
    adata_real=real_ct,
    de_pred=None,
    de_real=None,
    control_pert="non-targeting",
    pert_col="target",
    de_method="wilcoxon",
    num_threads=-1,
    batch_size=100,
    outdir="./cell-eval-outdir",
    allow_discrete=False,
    prefix=None,
    skip_de=False,
)

In [ ]:
from cell_eval impor

tensor([[1.5518, 1.0223, 0.4321,  ..., 0.0127, 0.2057, 0.7264],
        [0.7055, 0.7062, 0.7564,  ..., 0.0593, 0.0000, 1.0425],
        [0.7644, 1.2055, 0.3125,  ..., 0.0699, 0.3330, 0.8380],
        ...,
        [0.9589, 1.4247, 0.5020,  ..., 0.0650, 0.0365, 0.5431],
        [0.1778, 1.5413, 0.0811,  ..., 0.2136, 0.0000, 0.7665],
        [0.8655, 1.1622, 0.5321,  ..., 0.0669, 0.0123, 0.7398]],
       device='cuda:0')

In [164]:
torch.tensor(perturb_profile_dict['k562'][:, perturb_profile_dict['k562'].var['highly_variable']].X.toarray())

tensor([[0.4090, 0.6985, 1.1057,  ..., 0.0000, 0.6985, 2.5747],
        [0.8084, 0.8084, 1.0530,  ..., 0.0000, 0.8084, 2.4489],
        [0.5808, 0.5808, 0.0000,  ..., 0.5808, 0.5808, 1.5969],
        ...,
        [0.5166, 0.0000, 0.8555,  ..., 0.0000, 0.5166, 2.2099],
        [0.0000, 0.7911, 0.0000,  ..., 0.0000, 1.2272, 1.2272],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 2.1731]])

In [ ]:
def cal_metric_new()

In [167]:
matrix_dict = {}
for item in cell_type_list: 
    for type in cell_type_list:
        perturb_profile = torch.tensor(perturb_profile_dict[item][:, perturb_profile_dict[item].var['highly_variable']].X.toarray())
        matrix_dict[(item, type)] = sampler.cal_metric(perturb_profile, control_profile_dict[type][:, control_profile_dict[type].var['highly_variable']])

In [174]:
cell_type = 'hepg2'
cell_type_data = state_Data[state_Data.obs['cell_line']==cell_type]
matrix_dict_perturb = {}
gene_list = set(cell_type_data.obs['gene'])
gene_data = torch.tensor(cell_type_data[cell_type_data.obs['gene']=='non-targeting'][:, cell_type_data.var['highly_variable']].X.toarray())
for type in gene_list: 
    matrix_dict_perturb[('non-targeting', type)] = sampler.cal_metric(gene_data, cell_type_data[cell_type_data.obs['gene']==type][:, cell_type_data.var['highly_variable']])

In [ ]:
matrix_dict_perturb

{('non-targeting', 'RPS8'): (0.8290762901306152, 0.91209257),
 ('non-targeting', 'CCT6A'): (0.894419252872467, 0.9497758),
 ('non-targeting', 'non-targeting'): (1.0, 1.0),
 ('non-targeting', 'HGS'): (0.905807375907898, 0.95416015),
 ('non-targeting', 'POGZ'): (0.9838405847549438, 0.9919827),
 ('non-targeting', 'TRAPPC4'): (0.9801661968231201, 0.9903984),
 ('non-targeting', 'RIOK1'): (0.9225167036056519, 0.96107495),
 ('non-targeting', 'ATP2A2'): (0.9802168011665344, 0.9907773),
 ('non-targeting', 'PTCD3'): (0.9760352969169617, 0.9884068),
 ('non-targeting', 'POLR2B'): (0.8579238653182983, 0.9283825)}

In [168]:
matrix_dict

{('jurkat', 'jurkat'): (0.9882532954216003, 0.99437463),
 ('jurkat', 'k562'): (0.6175300478935242, 0.8205467),
 ('jurkat', 'rpe1'): (0.33593934774398804, 0.74484843),
 ('jurkat', 'hepg2'): (0.45138877630233765, 0.73489785),
 ('k562', 'jurkat'): (0.5508226156234741, 0.8077812),
 ('k562', 'k562'): (0.980702817440033, 0.99034506),
 ('k562', 'rpe1'): (-0.05782675743103027, 0.6352488),
 ('k562', 'hepg2'): (0.4683115482330322, 0.7432175),
 ('rpe1', 'jurkat'): (0.5514518618583679, 0.7467724),
 ('rpe1', 'k562'): (0.3809424638748169, 0.644668),
 ('rpe1', 'rpe1'): (0.9944376945495605, 0.9972677),
 ('rpe1', 'hepg2'): (0.7057881951332092, 0.84981686),
 ('hepg2', 'jurkat'): (0.4251304268836975, 0.73420304),
 ('hepg2', 'k562'): (0.4744534492492676, 0.7268021),
 ('hepg2', 'rpe1'): (0.5653258562088013, 0.8596075),
 ('hepg2', 'hepg2'): (0.9839951992034912, 0.9919827)}

In [157]:
cell_data = state_Data[(state_Data.obs['gene']=='POGZ')&(state_Data.obs['cell_line']=='hepg2')]
sampler.cal_metric(sample_comb_list['k562'], cell_data[:, cell_data.var['highly_variable']] )

(0.9664002656936646, 0.9851414)

In [141]:
sampler.cal_metric(sample_comb, state_Data[:37, state_Data.var['highly_variable']] )

(0.969586193561554, 0.9848137)

In [143]:
state_Data[:37, state_Data.var['highly_variable']].obs


,gem_group,gene,gene_id,transcript,gene_transcript,sgID_AB,mitopercent,UMI_count,z_gemgroup_UMI,cell_line,perturb_label,batch_label,cell_line_label
cell_barcode,,,,,,,,,,,,,
TCCCACACAAGTCCAT-20,20,non-targeting,non-targeting,non-targeting,10806_non-targeting_non-targeting_non-targeting,non-targeting_00318|non-targeting_00877,0.078166,12998.0,-0.126799,hepg2,10,19,0
AAGAACATCTAGTGTG-16,16,non-targeting,non-targeting,non-targeting,11012_non-targeting_non-targeting_non-targeting,non-targeting_01721|non-targeting_01433,0.099047,12388.0,0.759928,hepg2,10,15,0
GGGTATTAGGAACATT-42,42,non-targeting,non-targeting,non-targeting,11025_non-targeting_non-targeting_non-targeting,non-targeting_01813|non-targeting_00122,0.092756,12258.0,-0.521161,hepg2,10,41,0
TTGACCCCACGAAGAC-52,52,non-targeting,non-targeting,non-targeting,10951_non-targeting_non-targeting_non-targeting,non-targeting_01273|non-targeting_03357,0.131519,14918.0,0.026687,hepg2,10,51,0
GAGGGATCACTCCGGA-42,42,non-targeting,non-targeting,non-targeting,11271_non-targeting_non-targeting_non-targeting,non-targeting_03467|non-targeting_01317,0.022323,17336.0,0.131101,hepg2,10,41,0
AGGATAATCCCTCTCC-18,18,non-targeting,non-targeting,non-targeting,11208_non-targeting_non-targeting_non-targeting,non-targeting_02977|non-targeting_01146,0.096092,13383.0,0.638283,hepg2,10,17,0
GTGTTAGAGCATCTTG-8,8,non-targeting,non-targeting,non-targeting,10771_non-targeting_non-targeting_non-targeting,non-targeting_00164|non-targeting_00374,0.099374,18677.0,-0.044485,hepg2,10,7,0
CCAATGACATTGACAC-50,50,non-targeting,non-targeting,non-targeting,10967_non-targeting_non-targeting_non-targeting,non-targeting_01376|non-targeting_00354,0.078363,19716.0,0.358782,hepg2,10,49,0
TATATCCTCATGCAGT-1,1,non-targeting,non-targeting,non-targeting,10869_non-targeting_non-targeting_non-targeting,non-targeting_00693|non-targeting_02043,0.113725,15344.0,0.279445,hepg2,10,0,0


In [136]:
sample_comb.shape

torch.Size([37, 201])

In [132]:
state_Data[:37, state_Data.var['highly_variable']]

View of AnnData object with n_obs × n_vars = 37 × 201
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable'
    obsm: 'X_hvg'

In [131]:
test_data_gene1[:, state_Data.var['highly_variable']]

View of AnnData object with n_obs × n_vars = 37 × 201
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable'
    obsm: 'X_hvg'

In [130]:
sample_comb

tensor([[0.7282, 0.9856, 0.6788,  ..., 0.3601, 0.0000, 1.2772],
        [0.1632, 0.7973, 0.2425,  ..., 0.1355, 0.4455, 1.0979],
        [0.2565, 1.2333, 0.6216,  ..., 0.0000, 0.0631, 1.1449],
        ...,
        [1.0260, 1.0914, 0.3689,  ..., 0.0433, 0.0894, 1.1866],
        [0.6659, 1.0609, 0.2292,  ..., 0.0139, 0.2113, 1.1310],
        [0.3912, 1.1426, 0.3057,  ..., 0.2335, 0.0000, 1.1373]],
       device='cuda:0')

In [ ]:
gene_list = ['POGZ', 'RPS8']
for gene in gene_list:
    

In [105]:
#cosine similarity between different cell types
from sklearn.metrics.pairwise import cosine_similarity
cell_types = list(perturb_direction_dict.keys())
directions = torch.stack([perturb_direction_dict[ct] for ct in cell_types]).detach().cpu().numpy()
similarity_matrix = cosine_similarity(directions)

In [106]:
similarity_matrix

array([[ 1.0000001 ,  0.18834874, -0.03933998,  0.24443385],
       [ 0.18834874,  1.0000002 , -0.00648843,  0.01074759],
       [-0.03933998, -0.00648843,  1.        ,  0.09405403],
       [ 0.24443385,  0.01074759,  0.09405403,  0.99999994]],
      dtype=float32)

In [55]:
similarity_matrix

array([[ 0.99999994,  0.4493432 , -0.54073685,  0.3392582 ],
       [ 0.4493432 ,  0.99999976, -0.3435316 ,  0.12021162],
       [-0.54073685, -0.3435316 ,  1.0000001 , -0.64503545],
       [ 0.3392582 ,  0.12021162, -0.64503545,  1.0000001 ]],
      dtype=float32)

In [ ]:
test_gene = 

In [ ]:
test_ct = tuple(config['fewshot'].keys())[0].split('.')[1]
control_label = 'non-targeting'
test_perturb = tuple(config['fewshot'].items())[0][1]['test']
for 

In [ ]:
sampler.model.encoder.forward()

In [5]:
import torch

# 加载 checkpoint（如果显存有限可以用 map_location="cpu"）
ckpt = torch.load("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours/model.pt",
                  map_location="cpu")

# 1. 如果是直接保存的 state_dict
if isinstance(ckpt, dict) and all(isinstance(v, torch.Tensor) for v in ckpt.values()):
    keys = ckpt.keys()
# 2. 如果 checkpoint 是个 dict，里头有 'state_dict' 或类似 key
elif isinstance(ckpt, dict) and "state_dict" in ckpt:
    keys = ckpt["state_dict"].keys()
else:
    # 其他情况，先看一下顶层 keys
    print("checkpoint 顶层 keys:", ckpt.keys())
    # 假设第一个 key 对应的就是参数 dict
    first = next(iter(ckpt.values()))
    if isinstance(first, dict):
        keys = first.keys()
    else:
        raise ValueError("无法识别 checkpoint 格式，请手动检查结构")

# 打印所有参数名
print("模型包含的权重层／参数名称：")
for k in keys:
    print(k)

模型包含的权重层／参数名称：
time_embed.0.weight
time_embed.0.bias
time_embed.2.weight
time_embed.2.bias
mlp_blocks.0.fc1.weight
mlp_blocks.0.fc1.bias
mlp_blocks.0.fc2.weight
mlp_blocks.0.fc2.bias
mlp_blocks.0.layer_norm1.weight
mlp_blocks.0.layer_norm1.bias
mlp_blocks.0.layer_norm2.weight
mlp_blocks.0.layer_norm2.bias
mlp_blocks.0.time_dense.weight
mlp_blocks.0.time_dense.bias
mlp_blocks.1.fc1.weight
mlp_blocks.1.fc1.bias
mlp_blocks.1.fc2.weight
mlp_blocks.1.fc2.bias
mlp_blocks.1.layer_norm1.weight
mlp_blocks.1.layer_norm1.bias
mlp_blocks.1.layer_norm2.weight
mlp_blocks.1.layer_norm2.bias
mlp_blocks.1.time_dense.weight
mlp_blocks.1.time_dense.bias
mlp_blocks.2.fc1.weight
mlp_blocks.2.fc1.bias
mlp_blocks.2.fc2.weight
mlp_blocks.2.fc2.bias
mlp_blocks.2.layer_norm1.weight
mlp_blocks.2.layer_norm1.bias
mlp_blocks.2.layer_norm2.weight
mlp_blocks.2.layer_norm2.bias
mlp_blocks.2.time_dense.weight
mlp_blocks.2.time_dense.bias
input_layer.weight
input_layer.bias
output_layer.weight
output_layer.bias
